# Setup

Ensure all dependencies are installed.


In [ ]:
%pip install -U -r requirements.txt

  Cloning ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git (to revision main) to /tmp/pip-install-hqxg6667/m2p_d851e24dc487432cabd9c86da5523f23
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git' /tmp/pip-install-hqxg6667/m2p_d851e24dc487432cabd9c86da5523f23


Check if we have GPU support, and if not, warn the user.


In [2]:
import torch
import warnings

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    warnings.warn("CUDA is not available. Running on CPU.")

CUDA is available!


Load our dataset...


In [ ]:
import os
from models.data_sources import ModelNet40
from models.dataset import PointCloudDataset
from typing import List

train_folder = "data/train"
test_folder = "data/test"

train_dataset: PointCloudDataset = None
test_dataset: PointCloudDataset = None

if os.path.exists(train_folder) and os.path.exists(test_folder):
    print("Train and test data found. Reading data...")
    train_files: List[str] = []
    test_files: List[str] = []

    for root, dirs, files in os.walk(train_folder):
        for file in files:
            train_files.append(os.path.join(root, file))

    for root, dirs, files in os.walk(test_folder):
        for file in files:
            test_files.append(os.path.join(root, file))

    train_dataset = PointCloudDataset(train_files)
    test_dataset = PointCloudDataset(test_files)
    k = 40
else:
    classes, train_dataset, test_dataset = ModelNet40.download(
        npoints=2500, train_outdir=train_folder, test_outdir=test_folder
    )
    k = len(classes)

print(f"Training data: {len(train_dataset)} samples")
print(f"Test data size: {len(test_dataset)} samples")

Zip already downloaded.
Creating dataset...
Found 40 classes.


Processing data:   2%|█▋                                                                        | 285/12389 [04:15<10:46:09,  3.20s/it]

# Training

Train a new classifier or load a new pre-trained one:


In [3]:
from pointnet.train_classification import train_classification
from pointnet.model import PointNetCls

classifier_path = ""  # pointnet/classifier.pth

if classifier_path and os.path.exists(classifier_path):
    print(f"Loading existing model from {classifier_path}...")
    classifier = PointNetCls(k=len(train_dataset.classes))
    classifier.load_state_dict(torch.load(classifier_path, weights_only=True))
    print("Classifier loaded successfully.")
else:
    classifier = train_classification(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        k=k,
        epochs=25,
        outf="./pointnet/cls",
    )

Random Seed:  7870


AttributeError: 'PointCloudDataset' object has no attribute 'classes'

Run a simple classification to get a visualization of our classifier.


In [ ]:
import random
import torch
from utils.plot import plot_points

i = random.randint(0, len(test_dataset) - 1)

input_data = test_dataset[i][0].unsqueeze(0)
input_data = input_data.transpose(1, 2)

classifier.eval()

with torch.no_grad():
    output = classifier(input_data)


scores: torch.Tensor = output[0][0]
print("Scores:", scores.tolist())
print(
    f"Expected class: {test_dataset.classes[int(test_dataset[i][1])]} ({int(test_dataset[i][1])})",
)
print(
    f"Predicted class: {test_dataset.classes[scores.argmax().item()]} ({scores.argmax().item()})"
)

points = input_data[0].transpose(0, 1).numpy()
plot_points(points)

Train the segmenter:


In [ ]:
from pointnet.train_segmentation import train_segmentation

train_segmentation(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    epochs=25,
    class_choice=0,
    outf="./pointnet/seg",
)